In [26]:
from copy import copy, deepcopy
from random import sample
from collections import defaultdict
from pathlib import Path
import networkx as nx
import pickle
from itertools import permutations
import numpy as np
from random import sample
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from scipy.stats import spearmanr
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
import unicodedata
import re

Soricut_and_Och_2015の論文を追試するコード  
分散表現(Word Embedding WE)化は一旦Gloveで済ませ、
input  
分散表現  
学習用単語群  

output  
(word1, word2)を入力するとspearman_corrを返すclass 

を作る  

実装手順  
1. 語彙集合から全てのルール候補を抽出  
    a. prefix or suffixが6文字以下のルールを総当たりで調べる
2. 分散表現を学習  
3. ルール候補を評価  
    a. ルール毎に適用可能な(w1, w2)のsetS_rを求める  
    b. S_rを最大1000pairにサンプリング  
    c. S_rのpair(w, w')毎に、残り全てのpairに対してcosine距離と  
    w1+(w'-w)に対してw2が何番目に近いか, rankを計算  
    d. S_rのpair(w, w')毎に、hit_rateを計算  
    残り全pairに対して何%が100位以内に収まったか  
4. 同一のルールに対し、方向ベクトルを一般化  
    a. ルール毎に、最も多くの単語をsupportする, rankが100以下になる(w, w')を選択  
    b. S_rから(w, w')がsupportするpairを削除  
    c. (w, w')がsupportするpair数が10以下になるまでa→bを繰り返す  
    d. transformation: ('suffix':ε:d:↑dethrone)を求める
5. 4で求めたsupport setを選別  
    a. t_rank < 30, t_cosine > 0.5 
6. グラフを作成  
    a. 語彙集合全ての語彙をNodeとして格納  
    b. 選別に残った(w1, w2), weights=(rank, cosine)をedgeに追加
7. グラフを正規化  
    a. w2の方が単語数が多くなるようにedgeを刈り取り  
    b. それでも多重なら, rankが小さい方のedgeを残す  
    c. それでも多重なら、cosineが小さい方のedgeを残す  

Moprhを用いた評価手順  
入力: (w1, w2)  
出力: Spearman ρ correlation、-1 < ρ < 1
1. w1, w2がWEにあれば、分散表現を用いて計算  
2. wが片方しか無い場合、正規化したグラフでもう片方のwの分散表現を推定する  
    a. 存在しない方のw(仮にxとする)に全てのtransformationを適用しx'を求める  
    b. x'がWEに存在する単語になった場合、最も出現数の多い物を採用する  
    c. 採用したyとtransformationの分散表現を用いてxを計算する  
3. 計算したw1, w2で計算  

In [76]:
class Morph:
    
    
    def __init__(self, embedding):
        self.embedding = embedding
        self.rules = None
        self.transformations = None
        self.transformations_evaluated = None
        self.graph = None
        self.graph_normalized = None
        
    
    def unicodeToAscii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        
    # 大文字を全部小文字にする
    # 無駄な空白や文字じゃないやつを全部消す
    def normalizeString(self, s):
        s = self.unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r"", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r"", s)
        return s
    
    def extract_rule(self, word1, word2, max_len=6):
        # make rule_dict
        rules = defaultdict(list)
        # check suffix
        i = 1
        while (word1[:i] == word2[:i]):
            i += 1
        if i != 1and i > max(len(word1[i-1:]), len(word2[i-1:])) <= max_len:
            rules[('suffix', word1[i-1:], word2[i-1:])].append((word1, word2))
            
        # check prefix
        i = 1
        while (word1[-i:] == word2[-i:]):
            i += 1
        if i != 1 and i > max(len(word1[:-i+1]), len(word2[:-i+1])) <= max_len:
            rules[('prefix', word1[:-i+1], word2[:-i+1])].append((word1, word2))
        return rules
    
    def extract_rule_wrapped(self, args):
        return self.extract_rule(*args)
      
    def extract_rules(self, vocab, read=True, save=False, file_name=None):
        '''
        Extract candidate prefix/suffix rules from V.
        
        input
        vocab(set): set of words 
        real(bool):
        save(bool):  
        file_name: hogehoge.pickle    '../preprocessed/FILE_NAME'
        
        output
        self.rules(dict): 
            key(tuple): ('prefix/suffix', 'ing', 'ed')
            value(list): list of word_pair (word1, word2) supported by rule
        '''
        if file_name:
            path = Path.cwd().parent.joinpath('preprocessed/{}'.format(file_name))
        else:
            path = Path.cwd().parent.joinpath('preprocessed/rules.pickle')
        if path.exists() and read:
            with path.open('rb') as f:
                self.rules = pickle.load(f)
        else:
            rules_full = defaultdict(list)
            args_list = [args for args in tqdm(permutations(vocab, 2))]
            p = Pool()
            rules_list = p.map(self.extract_rule_wrapped, args_list)
            [rules_full.update(rules) for rules in rules_list]
            self.rules = rules_full
            del rules_full
            if save:
                with path.open('wb') as f:
                    pickle.dump(self.rules, f)
                
    def downsample_rules(self, n_sample=1000):
        '''
        downsample the number of word pair in suppor set of each rule.
        '''
        for key, value in self.rules.items():
            if len(value) >= n_sample:
                self.rules[key] = sample(value, n_sample)
    
    def is_word_pairs_similar(self, word_pair1, word_pair2, annoy_index=None, topn=10):
        '''
        Calculate if 
            word2_1 + (word1_2 - word1_1)
        is similar to word2_2
        
        input
            word_pair1(tuple): (word1_1, word1_2), used as direction vector
            word_pair2(tuple): (word2_1, word2_2)
            
        output
            bool
        '''
        closest_n = self.embedding.most_similar(
            positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]],
                                                                         topn=topn, indexer=annoy_index)
        for word, cos_sim in closest_n:
            if word == word_pair2[1]:
                return True
        return False
    
    def get_similarity_rank(self, word_pair1, word_pair2, topn=100):
        '''
        Calculate similarity rank and cosine similarity
        
        input
            word_pair1(tuple): direction vector
            word_pair2(tuple): word pair to calculate similarity
        return
            tuple (rank, cos_sim)
        '''
        closest_n = self.embedding.most_similar(
            positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
        for rank, (word, cos_sim) in enumerate(closest_n):
            if word == word_pair2[1]:
                return (rank+1, cos_sim) # add 1 to the highest rank to 1, not 0
        return (None, None)
    
    def index_vector(self, dimensions=300, save=False):
        '''
        make annoy_index which is used in function 'is_word_pairs_similar'
        Using annoy_index, execution may be slower than normal index
        '''
        path = Path.cwd().parent.joinpath('preprocessed/annoy.index')
        if path.exists():
            annoy_index = AnnoyIndexer()
            annoy_index.load(str(path))
            annoy_index.model = self.embedding
        else:
            annoy_index = AnnoyIndexer(self.embedding, dimensions)
            if save:
                annoy_index.save(str(path))
        return annoy_index
    
    def get_best_transformation(self, rule, support_set, topn=100, return_hit_rate=False):
        '''
        Get the most supportable transformation from the rule.
    
        input
            rule(tuple): (type, from, to)
            support_set(set): {(word1, word2), ...}
            topn(int): compute rank and cos_sim if rank is under topn
    
        return
            transformation(tuple): ('suffix', 'ing', 'ed', w, w')
            trans_support_set(set): {((word1, word2), rank, cosine), ...}
            hit_rate(real): optional                 
        '''
        # Calculate all the transformations
        transformations = defaultdict(tuple)
        for word_pair1 in support_set:
            transformation = rule + word_pair1  # tuple(type, from, to, w, w')
            trans_support_set = set()
            n_hit = 0
            for word_pair2 in support_set:
                if word_pair2 != word_pair1:
                    (rank, cos_sim) = self.get_similarity_rank(word_pair1, word_pair2, topn=topn)
                    trans_support_word = (word_pair2, rank, cos_sim)
                    if rank is not None:
                        n_hit += 1
                        trans_support_set.add(trans_support_word)
            # avoid 0 division
            hit_rate = n_hit / \
                len(trans_support_set) if len(trans_support_set) != 0 else 0
            transformations[transformation] = (hit_rate, trans_support_set)
    
        # Select hit_rate-highest one
        transformations_by_count = sorted(
            transformations.items(), key=lambda kv: kv[1][0], reverse=True)
        best_transformation, (best_hit_rate,
                              trans_support_set) = transformations_by_count[0]
    
        if return_hit_rate:
            return best_transformation, trans_support_set, best_hit_rate
        else:
            return best_transformation, trans_support_set

    def generate_transformations(self, topn=100, min_explain_count=10):
        '''
        Generate transformations and their support_sets
        
        return
            transformations(dict):
                key: transformation (type, from, to, word1, word2)
                value: transformation-support set {((word1, word2), rank, cosine), ...}
        '''
        transformations = defaultdict(dict)
        for rule, support_set in tqdm(self.rules.items()):
            support_set = set(support_set)
            while True:
                (tran, tran_support_set) = self.get_best_transformation(rule, support_set, topn=topn)
                if len(tran_support_set) < min_explain_count:
                    break
                transformations[tran] = tran_support_set
                tran_support_words = {word[0] for word in tran_support_set}
                support_set = support_set - {tran[3:5]} - tran_support_words
                if len(support_set) < min_explain_count:
                    break
        self.transformations = transformations
    
    def generate_trans(self, rule, support_set, topn, min_explain_count):
        trans_on_rule = defaultdict(dict)
        support_set = set(support_set)
        while True:
            (tran, tran_support_set) = self.get_best_transformation(
                rule, support_set, topn=topn)
            if len(tran_support_set) < min_explain_count:
                break
            trans_on_rule[tran] = tran_support_set
            tran_support_words = {word[0] for word in tran_support_set}
            support_set = support_set - {tran[3:5]} - tran_support_words
            if len(support_set) < min_explain_count:
                break
        return trans_on_rule

    def generate_trans_wrapped(self, args):
        return self.generate_trans(*args)

    def generate_transformations_parallel(self, topn=100, min_explain_count=10):
        '''
        Generate transformations and their support_sets in parallel
            
        return
            transformations(dict):
                key: transformation (type, from, to, word1, word2)
                value: transformation-support set {((word1, word2), rank, cosine), ...}
        '''
        transformations = defaultdict(dict)    
        args =[(rule, support_set, topn, min_explain_count) for rule, support_set in self.rules.items()]
        p = Pool() # processesを指定しなければ、cpuの指定出来る最大コア数を使う
        trans_list = p.map(self.generate_trans_wrapped, args)
        [transformations.update(trans) for trans in trans_list]
        self.transformations = transformations

    def evaluate_transformations(self, rank=30, cos_sim=0.5):
        '''
        Evaluate how well it passes a proximity test in embedding space.
        
        return:
            self.evaluated_transformations
        '''
        trans = deepcopy(self.transformations)
        for tran, support_set in tqdm(self.transformations.items()):
            for word in support_set:
                if word[1] > rank or word[2] < cos_sim: # cos_simの制約で結構落ちる
                    trans[tran].remove(word)
        self.transformations_evaluated = trans
        
    def build_graph(self, is_use_trans_evaluated=True):
        '''
        build Graph and add Nodes and Edges to self.graph
    
        input:
            is_use_trans_evaluated(bool)
        '''
        G = nx.MultiDiGraph()
        G.add_nodes_from(self.embedding.vocab.keys())
        if is_use_trans_evaluated:
            transformations = self.transformations_evaluated
        else:
            transformations = self.transformations
        for dw, support in tqdm(transformations.items()):
            # word_pair = dw so assign 0 for rank
            G.add_edge(dw[3], dw[4], transformation=dw, rank=0, cos_sim=1)
            for ((word1, word2), rank, cos_sim) in support:
                G.add_edge(word1, word2, transformation=dw, rank=rank, cos_sim=cos_sim)
        self.graph = G
        
    def normalize_graph(self):
        '''
        convert multi-directed self.graphraph to directed self.graphraph
        '''
        graph = deepcopy(self.graph)
        for node in tqdm(self.graph.nodes()):
            for neighbor in self.graph.neighbors(node):
                # if node and neighbor do not have multiple edges, continue
                if not (self.graph.has_edge(node, neighbor) and self.graph.has_edge(neighbor, node)):
                    continue
                # Delete one of the edges
                # edge w1→w2 considered only if count(w1) < count(w2)
                if self.embedding.vocab[node].count > self.embedding.vocab[neighbor].count:
                    graph.remove_edge(node, neighbor)
                    continue
                # chose the one with minimal rank
                if self.graph[node][neighbor][0]['rank'] > self.graph[neighbor][node][0]['rank']:
                    graph.remove_edge(node, neighbor)
                    continue
                # chose the one with the maximal cosine
                if self.graph[node][neighbor][0]['cos_sim'] < self.graph[neighbor][node][0]['cos_sim']:
                    graph.remove_edge(node, neighbor)
                    continue
            self.graph_normalized = graph
            
    def get_morph_trans(self, use_normalized_graph=True):
        '''
        Extract all transformations from the graph.
        
        input:
            use_normalized_graph(bool): use self.graph_normalized if True or use self.graph
        
        output:
            morph_trans: morphological transformations (type, from, to, w, w')
        '''
        morph_trans = set()
        if use_normalized_graph is True:
            for edge in self.graph_normalized.edges(data=True):
                morph_trans.add(edge[2]['transformation'])
        else:
            for edge in self.graph.edges(data=True):
                morph_trans.add(edge[2]['transformation'])
        return morph_trans

実際の使い方

ディレクトリ構造  
script  
 -このノートブック  
input  
 -GoogleNews  
preprocessed  
 -morph.rulesやmorph.transformationsなど一時的な計算結果を保存  
output  

In [84]:
embedding = KeyedVectors.load_word2vec_format('../input/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=1000)
morph = Morph(embedding)

In [86]:
normalizeString('The')

'the'

In [87]:
# あらかじめ入力する単語を綺麗にしておく
# 全て小文字にして、a-z以外の単語を取り除く
normalized_words = set([morph.normalizeString(word) for word in embedding.vocab.keys()])

In [88]:
morph.extract_rules(normalized_words, read=False, save=False)

In [82]:
morph.rules

defaultdict(list,
            {('prefix', 'a', 'we'): [('are', 'were')],
             ('prefix', 'a', 'mo'): [('are', 'more')],
             ('prefix', 'c', 'th'): [('can', 'than')],
             ('prefix', 'c', ''): [('can', 'an')],
             ('prefix', 'ot', ''): [('other', 'her')],
             ('suffix', 'w', 't'): [('now', 'not')],
             ('suffix', 'w', ''): [('now', 'no')],
             ('prefix', '', 'u'): [('s', 'us')],
             ('prefix', '', 'i'): [('s', 'is')],
             ('suffix', '', 'o'): [('s', 'so')],
             ('prefix', '', 'a'): [('s', 'as')],
             ('prefix', 'u', ''): [('us', 's')],
             ('prefix', 'u', 'i'): [('us', 'is')],
             ('suffix', 's', 'p'): [('us', 'up')],
             ('prefix', 'u', 'a'): [('us', 'as')],
             ('prefix', 'we', 'a'): [('were', 'are')],
             ('suffix', 're', ''): [('there', 'the')],
             ('prefix', 'we', 'mo'): [('were', 'more')],
             ('prefix', 'w', 'th'): [('wer

In [92]:
# limitを5000以上にすると計算量が爆発的に増えて処理が終わらなくなる
embedding = KeyedVectors.load_word2vec_format('../input/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=100000)
morph = Morph(word_vectors)
print('Extract rules...')
morph.extract_rules(embedding.vocab.keys(), read=True, save=False, file_name='rules_10000.pickle')
with open('../preprocessed/rules_100000.pickle', mode='wb') as f:
    pickle.dump(morph.rules, f)
print('Downsample rules...')
morph.downsample_rules()
print('Generate transformations...')
morph.generate_transformations_parallel(topn=10, min_explain_count=4)
with open('./preprocessed/transformations_100000.pickle', mode='wb') as f:
    pickle.dump(morph.transformations, f)
print('Evaluate transformations...')
morph.evaluate_transformations(rank=3, cos_sim=0.5)
print('Building graph...')
morph.build_graph(is_use_trans_evaluated=True)
print('Normalize graph...')
morph.normalize_graph()
with open('../preprocessed/normalized_graph_100000.pickle', mode='wb') as f:
    pickle.dump(morph.graph_normalized, f)

Extrace rules...


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Morphを用いたRare Wordsetの評価の仕方

In [4]:
embedding = KeyedVectors.load_word2vec_format('../input/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=100000)

In [6]:
morph = Morph(embedding)

In [7]:
with open('../preprocessed/rules_100000.pickle', mode='rb') as f:
    morph.rules = pickle.load(f)
    
with open('../preprocessed/transformations_100000.pickle', mode='rb') as f:
    morph.transformations = pickle.load(f)

In [8]:
print('Evaluate transformations...')
morph.evaluate_transformations(rank=3, cos_sim=0.5)
print('Building graph...')
morph.build_graph(is_use_trans_evaluated=True)
print('Normalize graph...')
morph.normalize_graph()

Evaluate transformations...



Building graph...



Normalize graph...


In [9]:
morph_trans = morph.get_morph_trans(use_normalized_graph=True)
embedding = morph.embedding

In [10]:
def apply_rule(rule, word):
    '''
    apply the transformation to the word
    rule: (type, from, to)   tran: (type, from, to, w, w')
    word: str
    '''
    if rule[0] == 'suffix' and word[::-1].startswith(rule[1][::-1]):
        return word[::-1].replace(rule[1][::-1], rule[2][::-1], 1)[::-1]
    elif rule[0] == 'prefix' and word.startswith(rule[1]):
        return word.replace(rule[1], rule[2], 1)
    else:
        return False

In [11]:
def get_oov_vector(trans, word, embedding):
    '''
    input
        trans: set of (type, from, to, w, w')
        word: str, not in V
        
    output:
        word_vec: vector or Nonr
        match: True or False
    '''
    nw = (None, None, 0) # (new_word, tran, count)
    for tran in trans:
        candidate_word = apply_rule(tran, word) # return word or False
        if candidate_word in embedding.vocab.keys():
            if embedding.vocab[candidate_word].count > nw[2]:
                nw = (candidate_word, tran, embedding.vocab[candidate_word].count)
    if nw[0]: # if new_word exists
        # print('Hit morph trans: {}→{}'.format(word, nw[0]))
        nwv = embedding[nw[0]] - (embedding[nw[1][4]] - embedding[nw[1][3]]) # nw: (new_word, (type, from, to, w, w'), count)
        return (nwv, True)
    else:
        return (None, False)

In [12]:
rw = pd.read_csv('../input/rw.txt', header=None,sep='\t')

In [13]:
rw.shape

(2034, 13)

In [15]:
%%time
corr_list = list() # 計算した相関係数を格納するlist
for index, row in rw.iterrows():
    w1, w2 = row[0], row[1]
    try:
        cor, p = spearmanr(embedding[w1], embedding[w2])
        corr_list.append(cor)
    except Exception as e:
        if w1 in embedding.vocab:
            w2_vec, match = get_oov_vector(morph_trans, w2, embedding)
            if match:
                cor, p = spearmanr(embedding[w1], w2_vec)
                corr_list.append(cor)
        elif w2 in embedding.vocab:
            w1_vec, match = get_oov_vector(morph_trans, w1, embedding)
            if match:
                cor, p = spearmanr(w1_vec, embedding[w2])
                corr_list.append(cor)

CPU times: user 3.02 s, sys: 76 ms, total: 3.1 s
Wall time: 1.55 s


In [16]:
print('Performance: {}'.format(np.mean(corr_list) * 100))
print('All pairs: {}'.format(rw.shape[0]))
print('Evaluated pairs: {}'.format(len(corr_list)))

Performance: 21.21148513236549
All pairs: 2034
Evaluated pairs: 1317


パフォーマンスが悪いのは多分分析に使用しているembeddingが小さいため  
limitを10000→100000にしたらPerformanceが18→21に上がった

In [18]:
corr_list = list()
for index, row in rw.iterrows():
    w1, w2 = row[0], row[1]
    try:
        cor, p = spearmanr(embedding[w1], embedding[w2])
        corr_list.append(cor)
    except:
        continue

In [19]:
print('Performance: {}'.format(np.mean(corr_list) * 100))
print('All pairs: {}'.format(rw.shape[0]))
print('Evaluated pairs: {}'.format(len(corr_list)))

Performance: 24.419332287358923
All pairs: 2034
Evaluated pairs: 782


Morphを使うことで、評価出来るペアの数は増えた  
一方で、平均パフォーマンスは下がった

In [20]:
%%time
corr_list = list() # 計算した相関係数を格納するlist
for index, row in rw.iterrows():
    w1, w2 = row[0], row[1]
    try:
        cor, p = spearmanr(embedding[w1], embedding[w2])
        # corr_list.append(cor)
    except Exception as e:
        if w1 in embedding.vocab:
            w2_vec, match = get_oov_vector(morph_trans, w2, embedding)
            if match:
                cor, p = spearmanr(embedding[w1], w2_vec)
                corr_list.append(cor)
        elif w2 in embedding.vocab:
            w1_vec, match = get_oov_vector(morph_trans, w1, embedding)
            if match:
                cor, p = spearmanr(w1_vec, embedding[w2])
                corr_list.append(cor)

CPU times: user 4.44 s, sys: 128 ms, total: 4.57 s
Wall time: 2.29 s


In [21]:
print('Performance: {}'.format(np.mean(corr_list) * 100))
print('All pairs: {}'.format(rw.shape[0]))
print('Evaluated pairs: {}'.format(len(corr_list)))

Performance: 16.52263190768349
All pairs: 2034
Evaluated pairs: 535


Morphで計算できたペアのみだと、Performanceは16.5  
デフォルトのembeddingと比べると精度は落ちる  